# 환경변수 설정

In [77]:
from dotenv import load_dotenv
import os
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv(override=True)

# 환경 변수에서 PINECONE_API_KEY를 가져옵니다.
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

# Pinecone 클라이언트 초기화

In [78]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

# 레코드 업서트 
- update + insert 데이터가 있으면 업데이트, 없으면 새로 삽입


In [79]:
# "embedding-3d"라는 이름의 인덱스를 초기화합니다.
index = pc.Index("lastproject")

# 인덱스의 통계 정보를 설명하는 메서드를 호출하고, 그 결과를 출력합니다.
# 인덱스의 통계 정보에는 인덱스에 저장된 벡터의 수, 차원, 메타데이터 등의 정보가 포함될 수 있습니다.
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'Industry': {'vector_count': 1571}},
 'total_vector_count': 1571,
 'vector_type': 'dense'}

- namespaces 가 비어 있음
- namespaces에 이름이 없으면 hash가 없기 때문에 서칭 속도가 늦어짐 / hast는 고유 숫자 값 즉, namespace 그룹에 속해 있지 않음녀 전체 공간을 다 뒤져야 한다는 뜻
- namespaces를 쓰면 속도가 높아짐

# langchain_openai 임베딩 객체 생성


In [80]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 클래스를 초기화합니다.
# OPENAI_API_KEY는 환경 변수에서 가져온 API 키입니다.
# embedding = OpenAIEmbeddings(model="text-embedding-3-small", api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# 텍스트분할

In [81]:
# 위키피이아 데이터 셋
# 허깅페이스의 데이터를 다운로드 받을 때 설치 필요
%pip install -qU datasets

Note: you may need to restart the kernel to use updated packages.


In [82]:
import pandas as pd

df1 = pd.read_csv("data/산업DB.v.0.3.csv")

for record in df1:
    print(record)

KRX 업종명
상세내용


In [83]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

df1['text'] = df1.apply(lambda row: f"{row['KRX 업종명']}: {row['상세내용']}", axis=1)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=30,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

chunks = []
for idx, text in enumerate(df1['text']):
    split_texts = splitter.split_text(text)
    for j, chunk in enumerate(split_texts):
        chunks.append({
            "id": f"{idx}-{j}",
            "KRX 업종명": df1.iloc[idx]['KRX 업종명'],
            "상세내용": chunk
        })

# 결과를 DataFrame으로 저장
chunk_df1 = pd.DataFrame(chunks)
chunk_df1

,id,KRX 업종명,상세내용
0,0-0,IT 서비스,"IT 서비스: 컴퓨터\n특정 고객의 요구에 의하여 주문형 소프트웨어를 자문, 개발 ..."
1,1-0,IT 서비스,"IT 서비스: 컴퓨터\n컴퓨터\n템을 기획, 설계하는 산업활동을 말한다. 이 사업체..."
2,2-0,IT 서비스,IT 서비스: 컴퓨터시설 관리업\n고객 사업장의 컴퓨터 시스템 및 자료 처리시설을...
3,3-0,IT 서비스,IT 서비스: 기타 정보 기술 및 컴퓨터 운영 관련 서비스업\n기타 컴...
4,3-1,IT 서비스,"J\n정 보 통 신 업\n정보서비스업\n자료 처리 및 데이터베이스 구축, 웹 ..."
...,...,...,...
1566,1202-0,화학,화학: 플라스틱 접착처리 제품 제조업\n플라스틱 제품 표면에 접착 물질을 처리...
1567,1203-0,화학,"화학: 플라스틱 적층, 도포 및 기타 표면처리 제품 제조업\n플라스틱 ..."
1568,1204-0,화학,"화학: 그 외 기타 플라스틱 제품 제조업\n개인, 가정, 식탁 및 ..."
1569,1204-1,화학,․덮개를 씌우지 않은 셀룰러 플라스틱 매트리스 제조(320)\n․재생 플라스틱제품 ...


In [ ]:
# index.delete(delete_all=True, namespace="Industry") # 인덱스 삭제

{}

In [ ]:
from uuid import uuid4
from tqdm.auto import tqdm
import time

batch_size = 100
texts = []
metas = []
count = 0

# chunk_df1 반복
for _, row in tqdm(chunk_df1.iterrows(), total=len(chunk_df1)):
    text_chunk = row["상세내용"]         # 실제 업로드할 텍스트 청크
    industry = row["KRX 업종명"]        # 업종명 (메타데이터 용)
    chunk_id = row["id"]               # 청크 ID

    # 메타데이터 구성 (검색 결과에 함께 나타나도록)
    metadata = {
        "chunk_id": chunk_id,
        "산업": industry,
        "상세내용" : chunk
    }

    texts.append(text_chunk)
    metas.append(metadata)
    count += 1

    # 일정 수만큼 모이면 업로드
    if count % batch_size == 0:
        ids = [str(uuid4()) for _ in range(len(texts))]
        embeddings = embedding.embed_documents(texts)
        index.upsert(vectors=zip(ids, embeddings, metas), namespace="Industry")
        texts, metas = [], []
        time.sleep(1)  # 과부하 방지

# 마지막 남은 것 업로드
if texts:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeddings = embedding.embed_documents(texts)
    index.upsert(vectors=zip(ids, embeddings, metas), namespace="Industry")


AttributeError: 'list' object has no attribute 'iterrows'